In [1]:
⎕IO ← 0            ⍝ Index origin is zero
]box on -style=max ⍝ Show boxes at max verbosity
]rows on           ⍝ Don't wrap long output lines
I←⌷⍨∘⊃⍨⍤0 99       ⍝ Sane indexing
assert ← {⍺ ← 'assertion failure' ⋄ 0∊⍵: ⍺ ⎕signal 8 ⋄ shy ← 0} ⍝ assert

┌→─────────────────┐
│Was OFF -style=max│
└──────────────────┘

┌→──────┐
│Was OFF│
└───────┘

## Part: visible trees

#### Example data

In [13]:
⎕ ← heights ← ⊃⎕NGET'ex1.txt'1
⎕ ← heights ← ↑⍎¨¨heights

┌→────────────────────────────────────────┐
│ ┌→────┐ ┌→────┐ ┌→────┐ ┌→────┐ ┌→────┐ │
│ │30373│ │25512│ │65332│ │33549│ │35390│ │
│ └─────┘ └─────┘ └─────┘ └─────┘ └─────┘ │
└∊────────────────────────────────────────┘

┌→────────┐
↓3 0 3 7 3│
│2 5 5 1 2│
│6 5 3 3 2│
│3 3 5 4 9│
│3 5 3 9 0│
└~────────┘

In [77]:
⍝ The basic algorithm (from the internet)
⌈⍀heights                       ⍝ Max scan downwards
¯1⍪¯1↓⌈⍀heights                 ⍝ Drop the last row and catenate -1 because we can always see the first row
heights>¯1⍪¯1↓⌈⍀heights         ⍝ Compare the heights. 1 = visible
find_visible ← {⍵>¯1⍪¯1↓⌈⍀⍵}

┌→────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
↓1 3 1 2 1 0 2 2 3 1 2 2 0 3 4 2 0 1 1 0 0 0 5 0 1 0 5 4 2 4 5 2 3 0 4 4 1 3 0 4 1 1 4 2 2 3 1 5 4 0 1 6 4 1 6 0 2 2 4 4 3 4 5 3 1 3 4 1 3 0 5 0 1 5 4 1 4 2 5 1 1 2 1 0 2 3 2 2 3 0 0 1 0 1 1 3 3 1 0│
│3 3 1 2 1 3 2 2 3 1 4 4 2 4 4 4 3 3 1 4 1 4 5 1 1 0 5 5 4 4 5 2 3 4 5 5 4 5 4 6 2 1 4 3 2 3 5 5 4 6 6 6 4 1 6 2 6 3 4 5 3 5 5 5 1 3 4 3 3 4 5 2 4 5 4 1 4 2 5 1 1 2 3 4 4 3 2 2 3 2 3 2 3 2 1 3 3 3 1│
│3 3 1 2 1 3 2 3 3 1 4 4 2 4 4 4 3 3 5 4 2 5 5 4 3 1 5 5 5 4 5 2 3 4 5 5 4 5 5 6 6 5 4 6 2 3 5 5 6 6 6 6 4 1 6 2 6 6 4 5 6 6 5 6 2 3 4 3 5 4 5 2 4 5 4 1 4 2 5 4 1 2 3 4 4 3 2 4 3 2 3 2 4 4 1 3 3 3 1│
│3 3 2 3 1 3 4 4 4 1 4 4 3 4 4 4 4 3 5 4 5 5 5 4 5 2 5 5 5 5 5 2 6 4 5 5 6 6 5 6 6 5 4 6 3 6 5 5 6 6 6 6 4 6 6 5 6 6 6 5 6 6 5 6 4 3 4 3 5 4 5 4 5 5 4 3 4 3 5 4 2 3 4 4 4 3 2 4 3 4 4 2 4 4 4 3 3 3 1│
│3 3 2 3 1 3 4 4 4 4 4 4 3 4 4 4 4 3 5 4 5 5 5 5 5 2 5 5 5 5 5 2 6 4 6 5 6 6 5 6 6 5 6 6 3 6 5 5 6 6 6 6 4 6 6 5 6 6 6 6 6 6 5 6 4 3 4 5 5 4 5 4 5 5 4 3 4 3 5 4 2 3 4 5 4 3 2 4 3 4 4 2 4 4 4 3 3 3 2│
│3 3 2 3 1 3 4 4 4 4 4 4 3 4 4 4 4 3 5 4 5 5 5 5 5 2 5 5 6 5 5 2 6 4 6 5 6 6 5 6 6 5 6 6 3 6 5 5 6 6 6 6 4 6 6 5 6 6 6 6 6 6 5 6 4 5 5 5 5 6 5 4 5 5 4 3 4 3 5 4 2 4 4 5 4 3 2 4 4 4 4 2 4 4 4 3 4 3 3│
│3 3 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 5 4 5 5 5 5 5 2 5 5 6 5 5 2 6 6 6 5 6 6 5 6 6 5 6 6 4 6 5 5 6 6 6 6 4 6 6 5 6 6 6 6 6 6 5 6 6 5 5 5 5 6 5 4 5 6 4 3 4 3 5 4 2 5 4 5 4 4 2 4 4 4 4 2 4 4 4 3 4 3 3│
│3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 4 5 5 5 5 5 2 5 5 6 5 5 2 6 6 6 5 6 6 5 6 6 5 6 6 4 6 6 5 6 6 7 6 5 6 7 5 6 6 6 6 6 6 5 6 6 5 6 5 5 6 5 4 5 6 5 3 4 3 5 4 2 5 5 5 4 4 3 4 5 4 4 2 4 4 4 3 4 3 3│
│3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 4 5 5 5 5 6 6 5 5 6 5 5 6 6 6 6 5 6 6 6 6 7 6 6 7 5 7 7 5 7 6 7 7 6 6 7 6 6 6 6 6 6 6 5 6 6 5 6 5 5 6 5 4 5 6 5 3 5 4 5 4 4 5 5 5 4 4 3 4 5 4 4 2 4 4 4 3 4 3 4│
│4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 4 5 5 5 5 6 6 5 5 6 5 5 6 6 6 6 6 6 6 6 6 7 6 7 7 5 7 7 5 7 6 7 7 6 6 7 7 6 6 6 7 7 6 6 6 6 5 6 5 5 6 5 6 6 6 5 3 5 4 5 4 4 5 5 5 4 4 4 5 5 4 4 4 4 4 4 3 4 3 4│
│4 4 4 4 4 4 4 4 4 4 4 4 4 5 4 4 5 5 5 4 6 5 5 5 6 6 5 6 6 5 6 6 6 6 6 6 6 6 6 6 7 6 7 7 5 7 7 5 7 6 7 7 7 6 7 7 6 6 7 7 7 7 6 6 6 5 7 5 5 6 5 6 6 6 5 3 5 6 5 5 6 5 5 5 5 4 4 5 5 4 4 4 4 4 4 3 4 3 4│
│4 4 4 4 4 4 4 4 4 5 4 4 5 5 4 4 5 5 5 6 6 5 5 5 6 6 5 6 6 5 6 6 6 6 6 6 6 7 6 6 7 7 7 7 5 7 7 5 7 6 7 7 7 6 7 7 6 6 7 7 7 7 6 6 6 5 7 5 5 6 5 6 6 6 5 6 5 6 5 5 6 5 5 5 5 4 4 5 5 4 4 4 4 4 4 3 4 3 4│
│4 4 4 4 4 4 4 5 4 5 4 4 5 5 4 4 5 5 5 6 6 6 5 5 6 6 5 6 6 5 6 6 6 6 6 6 6 7 6 6 7 7 7 7 5 7 7 7 7 6 7 7 7 6 7 7 6 6 7 7 7 7 6 6 6 7 7 5 5 6 5 6 6 6 5 6 5 6 6 5 6 5 5 5 5 4 4 5 5 4 4 4 4 4 4 3 4 3 4│
│4 4 4 4 4 4 4 5 5 5 4 4 5 5 4 4 5 5 5 6 6 6 5 5 6 6 5 6 6 5 7 6 6 7 6 6 7 7 6 6 7 7 7 7 5 7 7 7 7 6 7 7 7 6 7 7 6 6 7 7 7 7 7 6 6 7 7 5 7 6 6 6 6 6 5 6 5 6 6 5 6 5 5 5 5 5 4 5 5 4 4 4 4 4 4 3 4 3 4│
│4 4 4 4 4 4 5 5 5 5 4 4 5 5 5 4 5 5 5 6 6 6 5 6 6 6 5 6 6 7 7 6 6 7 6 6 7 7 6 7 7 7 7 7 5 7 7 7 8 7 7 7 8 7 7 7 7 7 7 7 7 7 7 6 6 7 7 6 7 6 6 6 6 6 6 6 5 6 6 5 6 5 6 5 6 5 4 5 5 4 4 4 4 4 4 3 4 3 4│
│4 4 4 4 4 4 5 5 5 5 4 5 5 5 5 5 5 5 5 6 6 6 5 6 6 6 7 7 7 7 7 6 6 7 6 6 7 7 6 7 7 7 8 7 5 7 7 7 8 7 8 7 8 7 7 7 7 7 7 7 7 7 7 6 6 7 7 6 7 6 6 6 6 7 6 6 5 6 6 6 6 5 6 5 6 5 6 5 5 4 4 4 4 4 5 3 4 3 4│
│4 4 4 4 4 4 5 5 5 5 4 5 5 5 5 5 5 6 5 6 6 6 5 6 6 6 7 7 7 7 7 6 7 7 6 6 7 7 6 8 7 7 8 8 6 8 7 7 8 7 8 7 8 7 8 7 7 7 7 7 7 7 7 6 6 7 7 7 7 6 6 6 7 7 6 6 7 6 6 6 6 5 6 5 6 5 6 5 5 4 4 4 4 4 5 3 4 4 4│
│4 4 4 4 5 4 5 5 5 5 5 5 6 6 5 5 5 6 5 6 6 6 5 6 6 6 7 7 7 7 7 6 7 7 6 6 7 8 6 8 7 7 8 8 6 8 7 7 8 7 8 7 8 7 8 7 7 7 8 7 7 7 7 6 6 7 7 7 7 6 6 6 7 7 6 6 7 6 6 6 6 6 6 5 6 5 6 5 5 4 4 4 4 5 5 5 4 4 4│
│4 4 5 5 5 4 5 5 5 5 5 5 6 6 5 5 5 6 5 6 6 7 6 6 6 6 7 7 7 7 7 6 7 7 6 6 7 8 6 8 7 7 8 8 6 8 7 7 8 7 8 7 8 8 8 7 7 7 8 7 7 7 8 6 6 7 7 7 7 7 7 6 7 7 7 6 7 6 6 7 6 6 6 5 6 5 6 5 5 4 5 4 4 5 5 5 4 4 4│


┌→───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
↓¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1 ¯1│
│ 1 3 1 2 1 0 2 2 3 1 2 2 0 3 4 2 0 1 1 0 0 0 5 0 1 0 5 4 2 4 5 2 3 0 4 4 1 3 0 4 1 1 4 2 2 3 1 5 4 0 1 6 4 1 6 0 2 2 4 4 3 4 5 3 1 3 4 1 3 0 5 0 1 5 4 1 4 2 5 1 1 2 1 0 2 3 2 2 3 0 0 1 0 1 1 3 3 1 0│
│ 3 3 1 2 1 3 2 2 3 1 4 4 2 4 4 4 3 3 1 4 1 4 5 1 1 0 5 5 4 4 5 2 3 4 5 5 4 5 4 6 2 1 4 3 2 3 5 5 4 6 6 6 4 1 6 2 6 3 4 5 3 5 5 5 1 3 4 3 3 4 5 2 4 5 4 1 4 2 5 1 1 2 3 4 4 3 2 2 3 2 3 2 3 2 1 3 3 3 1│
│ 3 3 1 2 1 3 2 3 3 1 4 4 2 4 4 4 3 3 5 4 2 5 5 4 3 1 5 5 5 4 5 2 3 4 5 5 4 5 5 6 6 5 4 6 2 3 5 5 6 6 6 6 4 1 6 2 6 6 4 5 6 6 5 6 2 3 4 3 5 4 5 2 4 5 4 1 4 2 5 4 1 2 3 4 4 3 2 4 3 2 3 2 4 4 1 3 3 3 1│
│ 3 3 2 3 1 3 4 4 4 1 4 4 3 4 4 4 4 3 5 4 5 5 5 4 5 2 5 5 5 5 5 2 6 4 5 5 6 6 5 6 6 5 4 6 3 6 5 5 6 6 6 6 4 6 6 5 6 6 6 5 6 6 5 6 4 3 4 3 5 4 5 4 5 5 4 3 4 3 5 4 2 3 4 4 4 3 2 4 3 4 4 2 4 4 4 3 3 3 1│
│ 3 3 2 3 1 3 4 4 4 4 4 4 3 4 4 4 4 3 5 4 5 5 5 5 5 2 5 5 5 5 5 2 6 4 6 5 6 6 5 6 6 5 6 6 3 6 5 5 6 6 6 6 4 6 6 5 6 6 6 6 6 6 5 6 4 3 4 5 5 4 5 4 5 5 4 3 4 3 5 4 2 3 4 5 4 3 2 4 3 4 4 2 4 4 4 3 3 3 2│
│ 3 3 2 3 1 3 4 4 4 4 4 4 3 4 4 4 4 3 5 4 5 5 5 5 5 2 5 5 6 5 5 2 6 4 6 5 6 6 5 6 6 5 6 6 3 6 5 5 6 6 6 6 4 6 6 5 6 6 6 6 6 6 5 6 4 5 5 5 5 6 5 4 5 5 4 3 4 3 5 4 2 4 4 5 4 3 2 4 4 4 4 2 4 4 4 3 4 3 3│
│ 3 3 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 5 4 5 5 5 5 5 2 5 5 6 5 5 2 6 6 6 5 6 6 5 6 6 5 6 6 4 6 5 5 6 6 6 6 4 6 6 5 6 6 6 6 6 6 5 6 6 5 5 5 5 6 5 4 5 6 4 3 4 3 5 4 2 5 4 5 4 4 2 4 4 4 4 2 4 4 4 3 4 3 3│
│ 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 4 5 5 5 5 5 2 5 5 6 5 5 2 6 6 6 5 6 6 5 6 6 5 6 6 4 6 6 5 6 6 7 6 5 6 7 5 6 6 6 6 6 6 5 6 6 5 6 5 5 6 5 4 5 6 5 3 4 3 5 4 2 5 5 5 4 4 3 4 5 4 4 2 4 4 4 3 4 3 3│
│ 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 4 5 5 5 5 6 6 5 5 6 5 5 6 6 6 6 5 6 6 6 6 7 6 6 7 5 7 7 5 7 6 7 7 6 6 7 6 6 6 6 6 6 6 5 6 6 5 6 5 5 6 5 4 5 6 5 3 5 4 5 4 4 5 5 5 4 4 3 4 5 4 4 2 4 4 4 3 4 3 4│
│ 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 4 5 5 5 5 6 6 5 5 6 5 5 6 6 6 6 6 6 6 6 6 7 6 7 7 5 7 7 5 7 6 7 7 6 6 7 7 6 6 6 7 7 6 6 6 6 5 6 5 5 6 5 6 6 6 5 3 5 4 5 4 4 5 5 5 4 4 4 5 5 4 4 4 4 4 4 3 4 3 4│
│ 4 4 4 4 4 4 4 4 4 4 4 4 4 5 4 4 5 5 5 4 6 5 5 5 6 6 5 6 6 5 6 6 6 6 6 6 6 6 6 6 7 6 7 7 5 7 7 5 7 6 7 7 7 6 7 7 6 6 7 7 7 7 6 6 6 5 7 5 5 6 5 6 6 6 5 3 5 6 5 5 6 5 5 5 5 4 4 5 5 4 4 4 4 4 4 3 4 3 4│
│ 4 4 4 4 4 4 4 4 4 5 4 4 5 5 4 4 5 5 5 6 6 5 5 5 6 6 5 6 6 5 6 6 6 6 6 6 6 7 6 6 7 7 7 7 5 7 7 5 7 6 7 7 7 6 7 7 6 6 7 7 7 7 6 6 6 5 7 5 5 6 5 6 6 6 5 6 5 6 5 5 6 5 5 5 5 4 4 5 5 4 4 4 4 4 4 3 4 3 4│
│ 4 4 4 4 4 4 4 5 4 5 4 4 5 5 4 4 5 5 5 6 6 6 5 5 6 6 5 6 6 5 6 6 6 6 6 6 6 7 6 6 7 7 7 7 5 7 7 7 7 6 7 7 7 6 7 7 6 6 7 7 7 7 6 6 6 7 7 5 5 6 5 6 6 6 5 6 5 6 6 5 6 5 5 5 5 4 4 5 5 4 4 4 4 4 4 3 4 3 4│
│ 4 4 4 4 4 4 4 5 5 5 4 4 5 5 4 4 5 5 5 6 6 6 5 5 6 6 5 6 6 5 7 6 6 7 6 6 7 7 6 6 7 7 7 7 5 7 7 7 7 6 7 7 7 6 7 7 6 6 7 7 7 7 7 6 6 7 7 5 7 6 6 6 6 6 5 6 5 6 6 5 6 5 5 5 5 5 4 5 5 4 4 4 4 4 4 3 4 3 4│
│ 4 4 4 4 4 4 5 5 5 5 4 4 5 5 5 4 5 5 5 6 6 6 5 6 6 6 5 6 6 7 7 6 6 7 6 6 7 7 6 7 7 7 7 7 5 7 7 7 8 7 7 7 8 7 7 7 7 7 7 7 7 7 7 6 6 7 7 6 7 6 6 6 6 6 6 6 5 6 6 5 6 5 6 5 6 5 4 5 5 4 4 4 4 4 4 3 4 3 4│
│ 4 4 4 4 4 4 5 5 5 5 4 5 5 5 5 5 5 5 5 6 6 6 5 6 6 6 7 7 7 7 7 6 6 7 6 6 7 7 6 7 7 7 8 7 5 7 7 7 8 7 8 7 8 7 7 7 7 7 7 7 7 7 7 6 6 7 7 6 7 6 6 6 6 7 6 6 5 6 6 6 6 5 6 5 6 5 6 5 5 4 4 4 4 4 5 3 4 3 4│
│ 4 4 4 4 4 4 5 5 5 5 4 5 5 5 5 5 5 6 5 6 6 6 5 6 6 6 7 7 7 7 7 6 7 7 6 6 7 7 6 8 7 7 8 8 6 8 7 7 8 7 8 7 8 7 8 7 7 7 7 7 7 7 7 6 6 7 7 7 7 6 6 6 7 7 6 6 7 6 6 6 6 5 6 5 6 5 6 5 5 4 4 4 

┌→────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
↓1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1│
│1 0 0 0 0 1 0 0 0 0 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 0 1 1 0 0 0 0 1 1 1 1 1 1 1 1 0 0 1 0 0 1 0 0 1 1 0 0 0 0 1 1 1 0 1 0 1 0 1 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 1 1 1 1 0 0 0 1 1│
│0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 1 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0│
│0 0 1 1 0 0 1 1 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 1 0 0 0 1 0 0 1 0 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0│
│0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1│
│0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1│
│0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0│
│0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0│
│0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 1 1 1 1 0 1 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1│
│1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0│
│0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0│
│0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0│
│0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0│
│0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0│
│0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0│
│0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0│
│0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0│
│0 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0│
│0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0│


In [41]:
⍝ Use rotation to do it for all fours direction
4⍴⊂heights
{⌽⍉⍵}\4⍴⊂heights
rotations ← {{⌽⍉⍵}\4⍴⊂⍵}
find_visible¨{⌽⍉⍵}\4⍴⊂heights

┌→────────────────────────────────────────────────┐
│ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ │
│ ↓3 0 3 7 3│ ↓3 0 3 7 3│ ↓3 0 3 7 3│ ↓3 0 3 7 3│ │
│ │2 5 5 1 2│ │2 5 5 1 2│ │2 5 5 1 2│ │2 5 5 1 2│ │
│ │6 5 3 3 2│ │6 5 3 3 2│ │6 5 3 3 2│ │6 5 3 3 2│ │
│ │3 3 5 4 9│ │3 3 5 4 9│ │3 3 5 4 9│ │3 3 5 4 9│ │
│ │3 5 3 9 0│ │3 5 3 9 0│ │3 5 3 9 0│ │3 5 3 9 0│ │
│ └~────────┘ └~────────┘ └~────────┘ └~────────┘ │
└∊────────────────────────────────────────────────┘

┌→────────────────────────────────────────────────┐
│ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ │
│ ↓3 0 3 7 3│ ↓3 3 6 2 3│ ↓0 9 3 5 3│ ↓3 2 2 9 0│ │
│ │2 5 5 1 2│ │5 3 5 5 0│ │9 4 5 3 3│ │7 1 3 4 9│ │
│ │6 5 3 3 2│ │3 5 3 5 3│ │2 3 3 5 6│ │3 5 3 5 3│ │
│ │3 3 5 4 9│ │9 4 3 1 7│ │2 1 5 5 2│ │0 5 5 3 5│ │
│ │3 5 3 9 0│ │0 9 2 2 3│ │3 7 3 0 3│ │3 2 6 3 3│ │
│ └~────────┘ └~────────┘ └~────────┘ └~────────┘ │
└∊────────────────────────────────────────────────┘

┌→────────────────────────────────────────────────┐
│ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ │
│ ↓1 1 1 1 1│ ↓1 1 1 1 1│ ↓1 1 1 1 1│ ↓1 1 1 1 1│ │
│ │0 1 1 0 0│ │1 0 0 1 0│ │1 0 1 0 0│ │1 0 1 0 1│ │
│ │1 0 0 0 0│ │0 1 0 0 0│ │0 0 0 0 1│ │0 1 0 0 0│ │
│ │0 0 0 0 1│ │1 0 0 0 1│ │0 0 0 0 0│ │0 0 1 0 0│ │
│ │0 0 0 1 0│ │0 1 0 0 0│ │0 0 0 0 0│ │0 0 1 0 0│ │
│ └~────────┘ └~────────┘ └~────────┘ └~────────┘ │
└∊────────────────────────────────────────────────┘

In [98]:
⍝ Need to move to the same orientation
rotations heights                ⍝ Rotate
↑rotations¨rotations heights     ⍝ Get the rotations for each rotation
(0 3)(1 2)(2 1)(3 0)I↑rotations¨rotations heights     ⍝ Pick out a set of the same rotations
get_same_orientation ← {(0 3)(1 2)(2 1)(3 0)I↑rotations¨⍵}
{{⌽⍉⍵}\⌽¨{⌽⍉⍵}\4⍴⊂⍵}heights     ⍝ Internet alternative - doesn't return to original orientation but all end up in the same orientation

┌→────────────────────────────────────────────────┐
│ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ │
│ ↓3 0 3 7 3│ ↓3 3 6 2 3│ ↓0 9 3 5 3│ ↓3 2 2 9 0│ │
│ │2 5 5 1 2│ │5 3 5 5 0│ │9 4 5 3 3│ │7 1 3 4 9│ │
│ │6 5 3 3 2│ │3 5 3 5 3│ │2 3 3 5 6│ │3 5 3 5 3│ │
│ │3 3 5 4 9│ │9 4 3 1 7│ │2 1 5 5 2│ │0 5 5 3 5│ │
│ │3 5 3 9 0│ │0 9 2 2 3│ │3 7 3 0 3│ │3 2 6 3 3│ │
│ └~────────┘ └~────────┘ └~────────┘ └~────────┘ │
└∊────────────────────────────────────────────────┘

┌→────────────────────────────────────────────────┐
↓ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ │
│ ↓3 0 3 7 3│ ↓3 3 6 2 3│ ↓0 9 3 5 3│ ↓3 2 2 9 0│ │
│ │2 5 5 1 2│ │5 3 5 5 0│ │9 4 5 3 3│ │7 1 3 4 9│ │
│ │6 5 3 3 2│ │3 5 3 5 3│ │2 3 3 5 6│ │3 5 3 5 3│ │
│ │3 3 5 4 9│ │9 4 3 1 7│ │2 1 5 5 2│ │0 5 5 3 5│ │
│ │3 5 3 9 0│ │0 9 2 2 3│ │3 7 3 0 3│ │3 2 6 3 3│ │
│ └~────────┘ └~────────┘ └~────────┘ └~────────┘ │
│ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ │
│ ↓3 3 6 2 3│ ↓0 9 3 5 3│ ↓3 2 2 9 0│ ↓3 0 3 7 3│ │
│ │5 3 5 5 0│ │9 4 5 3 3│ │7 1 3 4 9│ │2 5 5 1 2│ │
│ │3 5 3 5 3│ │2 3 3 5 6│ │3 5 3 5 3│ │6 5 3 3 2│ │
│ │9 4 3 1 7│ │2 1 5 5 2│ │0 5 5 3 5│ │3 3 5 4 9│ │
│ │0 9 2 2 3│ │3 7 3 0 3│ │3 2 6 3 3│ │3 5 3 9 0│ │
│ └~────────┘ └~────────┘ └~────────┘ └~────────┘ │
│ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ │
│ ↓0 9 3 5 3│ ↓3 2 2 9 0│ ↓3 0 3 7 3│ ↓3 3 6 2 3│ │
│ │9 4 5 3 3│ │7 1 3 4 9│ │2 5 5 1 2│ │5 3 5 5 0│ │
│ │2 3 3 5 6│ │3 5 3 5 3│ │6 5 3 3 2│ │3 5 3 5 3│ │
│ │2 1 5 5 2│ │0 5 5 3 5│ │3 3 5 4 9│ │9 4 3 1 7│ │
│ │3 7 3 0 3│ │3 2 6 3 3│ │3 5 3 9 0│ │0 9 2 2 3│ │
│ └~────────┘ └~────────┘ └~────────┘ └~────────┘ │
│ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ │
│ ↓3 2 2 9 0│ ↓3 0 3 7 3│ ↓3 3 6 2 3│ ↓0 9 3 5 3│ │
│ │7 1 3 4 9│ │2 5 5 1 2│ │5 3 5 5 0│ │9 4 5 3 3│ │
│ │3 5 3 5 3│ │6 5 3 3 2│ │3 5 3 5 3│ │2 3 3 5 6│ │
│ │0 5 5 3 5│ │3 3 5 4 9│ │9 4 3 1 7│ │2 1 5 5 2│ │
│ │3 2 6 3 3│ │3 5 3 9 0│ │0 9 2 2 3│ │3 7 3 0 3│ │
│ └~────────┘ └~────────┘ └~────────┘ └~────────┘ │
└∊────────────────────────────────────────────────┘

┌→────────────────────────────────────────────────┐
│ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ │
│ ↓3 2 2 9 0│ ↓3 2 2 9 0│ ↓3 2 2 9 0│ ↓3 2 2 9 0│ │
│ │7 1 3 4 9│ │7 1 3 4 9│ │7 1 3 4 9│ │7 1 3 4 9│ │
│ │3 5 3 5 3│ │3 5 3 5 3│ │3 5 3 5 3│ │3 5 3 5 3│ │
│ │0 5 5 3 5│ │0 5 5 3 5│ │0 5 5 3 5│ │0 5 5 3 5│ │
│ │3 2 6 3 3│ │3 2 6 3 3│ │3 2 6 3 3│ │3 2 6 3 3│ │
│ └~────────┘ └~────────┘ └~────────┘ └~────────┘ │
└∊────────────────────────────────────────────────┘

┌→────────────────────────────────────────────────┐
│ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ │
│ ↓3 7 3 0 3│ ↓3 7 3 0 3│ ↓3 7 3 0 3│ ↓3 7 3 0 3│ │
│ │2 1 5 5 2│ │2 1 5 5 2│ │2 1 5 5 2│ │2 1 5 5 2│ │
│ │2 3 3 5 6│ │2 3 3 5 6│ │2 3 3 5 6│ │2 3 3 5 6│ │
│ │9 4 5 3 3│ │9 4 5 3 3│ │9 4 5 3 3│ │9 4 5 3 3│ │
│ │0 9 3 5 3│ │0 9 3 5 3│ │0 9 3 5 3│ │0 9 3 5 3│ │
│ └~────────┘ └~────────┘ └~────────┘ └~────────┘ │
└∊────────────────────────────────────────────────┘

┌→────────────────────────┐
↓ ┌→────────┐ ┌→────────┐ │
│ ↓3 3 6 2 3│ ↓3 3 6 2 3│ │
│ │5 3 5 5 0│ │5 3 5 5 0│ │
│ │3 5 3 5 3│ │3 5 3 5 3│ │
│ │9 4 3 1 7│ │9 4 3 1 7│ │
│ │0 9 2 2 3│ │0 9 2 2 3│ │
│ └~────────┘ └~────────┘ │
│ ┌→────────┐ ┌→────────┐ │
│ ↓0 9 3 5 3│ ↓0 9 3 5 3│ │
│ │9 4 5 3 3│ │9 4 5 3 3│ │
│ │2 3 3 5 6│ │2 3 3 5 6│ │
│ │2 1 5 5 2│ │2 1 5 5 2│ │
│ │3 7 3 0 3│ │3 7 3 0 3│ │
│ └~────────┘ └~────────┘ │
└∊────────────────────────┘

In [64]:
get_same_orientation find_visible¨rotations heights
∨⌿get_same_orientation find_visible¨rotations heights                 ⍝ Reduce using 'or'
,⊃∨⌿get_same_orientation find_visible¨rotations heights               ⍝ Flatten
+/,⊃∨⌿get_same_orientation find_visible¨rotations heights             ⍝ Count trees

┌→────────────────────────────────────────────────┐
│ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ │
│ ↓1 0 0 1 0│ ↓0 0 0 1 0│ ↓0 0 0 1 1│ ↓1 1 1 1 1│ │
│ │1 0 0 0 1│ │1 0 0 0 1│ │0 0 0 0 1│ │1 0 1 0 1│ │
│ │1 1 0 0 0│ │0 0 0 1 0│ │0 0 0 1 1│ │0 1 0 0 0│ │
│ │1 1 0 0 0│ │0 1 0 0 1│ │0 0 0 0 1│ │0 0 1 0 0│ │
│ │1 0 1 0 0│ │1 1 1 1 1│ │0 0 1 0 1│ │0 0 1 0 0│ │
│ └~────────┘ └~────────┘ └~────────┘ └~────────┘ │
└∊────────────────────────────────────────────────┘

┌─────────────┐
│ ┌→────────┐ │
│ ↓1 1 1 1 1│ │
│ │1 0 1 0 1│ │
│ │1 1 0 1 1│ │
│ │1 1 1 0 1│ │
│ │1 1 1 1 1│ │
│ └~────────┘ │
└∊────────────┘

┌→────────────────────────────────────────────────┐
│1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1│
└~────────────────────────────────────────────────┘

21

### Real data

In [72]:
heights ← ↑⍎¨¨⊃⎕NGET'input.txt'1
assert 1647 = +/,⊃∨⌿get_same_orientation find_visible¨rotations heights

In [78]:
⍝ Internet solution
h←⍎¨↑⊃⎕nget'input.txt'1
f←{↑{⌽⍉⍵}\⌽⍺⍺¨{⌽⍉⍵}\4⍴⊂⍵}
assert 1647 = +/,∨⌿(⊢>¯1⍪¯1↓⌈⍀)f h

## Part 2: furthest view from a tree

In [83]:
⎕ ← heights ← ⍎¨↑⊃⎕nget'ex1.txt'1

┌→────────┐
↓3 0 3 7 3│
│2 5 5 1 2│
│6 5 3 3 2│
│3 3 5 4 9│
│3 5 3 9 0│
└~────────┘

In [80]:
,⍀heights

┌→────────────────────────────────────────────────────────────┐
↓ │
│ 3 0 3 7 3 │
│ │
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ │
│ │3 2│ │0 5│ │3 5│ │7 1│ │3 2│ │
│ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ │
│ ┌→────┐ ┌→────┐ ┌→────┐ ┌→────┐ ┌→────┐ │
│ │3 2 6│ │0 5 5│ │3 5 3│ │7 1 3│ │3 2 2│ │
│ └~────┘ └~────┘ └~────┘ └~────┘ └~────┘ │
│ ┌→──────┐ ┌→──────┐ ┌→──────┐ ┌→──────┐ ┌→──────┐ │
│ │3 2 6 3│ │0 5 5 3│ │3 5 3 5│ │7 1 3 4│ │3 2 2 9│ │
│ └~──────┘ └~──────┘ └~──────┘ └~──────┘ └~──────┘ │
│ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ │
│ │3 2 6 3 3│ │0 5 5 3 5│ │3 5 3 5 3│ │7 1 3 4 9│ │3 2 2 9 0│ │
│ └~────────┘ └~────────┘ └~────────┘ └~────────┘ └~────────┘ │
└∊────────────────────────────────────────────────────────────┘

In [86]:
heights{⍺>⍵}¨,⍀heights

┌→────────────────────────────────────────────────────────────┐
↓ │
│ 0 0 0 0 0 │
│ │
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ │
│ │0 0│ │1 0│ │1 0│ │0 0│ │0 0│ │
│ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ │
│ ┌→────┐ ┌→────┐ ┌→────┐ ┌→────┐ ┌→────┐ │
│ │1 1 0│ │1 0 0│ │0 0 0│ │0 1 0│ │0 0 0│ │
│ └~────┘ └~────┘ └~────┘ └~────┘ └~────┘ │
│ ┌→──────┐ ┌→──────┐ ┌→──────┐ ┌→──────┐ ┌→──────┐ │
│ │0 1 0 0│ │1 0 0 0│ │1 0 1 0│ │0 1 1 0│ │1 1 1 0│ │
│ └~──────┘ └~──────┘ └~──────┘ └~──────┘ └~──────┘ │
│ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ │
│ │0 1 0 0 0│ │1 0 0 1 0│ │0 0 0 0 0│ │1 1 1 1 0│ │0 0 0 0 0│ │
│ └~────────┘ └~────────┘ └~────────┘ └~────────┘ └~────────┘ │
└∊────────────────────────────────────────────────────────────┘